In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pickle
import os 
import numpy as np
import getpass
import os


input_test_file = "/home/catsudon/nvidia/battery_timeserie/resources/raw/test_bat_data.pkl"

with open(input_test_file, "rb") as file:
    test_dict = pickle.load(file)
    
for battery_id in test_dict:
    trimmed_arr = np.trim_zeros(test_dict[battery_id]['q_d_n'], 'b')
    test_dict[battery_id]['trimmed_q_d_n'] = list(trimmed_arr)
    
last_k_th_cycles_list = [10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# calculate slope from first cycle to the last cycle
for battery_id in test_dict:
    q_d_n_values  = test_dict[battery_id]['trimmed_q_d_n']
    cycle = test_dict[battery_id]['cycle']
    slope = (q_d_n_values[-1] - q_d_n_values[0]) / cycle
    test_dict[battery_id]['slope_all_cycles'] = slope
    
for k in last_k_th_cycles_list:
    for battery_id in test_dict:
        q_d_n_values  = test_dict[battery_id]['trimmed_q_d_n']
        cycle = test_dict[battery_id]['cycle']
        if cycle > k:
            slope = (q_d_n_values[-1] - q_d_n_values[-k]) / k
        else:
            slope = np.nan
        test_dict[battery_id][f'slope_last_{k}_cycles'] = slope
        
#calculate a mean gradient of the last k cycles using numpy gradient
for k in last_k_th_cycles_list:
    for battery_id in test_dict:
        q_d_n_values  = test_dict[battery_id]['trimmed_q_d_n']
        cycle = test_dict[battery_id]['cycle']
        if cycle > k:
            slope = np.gradient(q_d_n_values[-k:], 1)
            mean_slope = np.mean(slope)
        else:
            mean_slope = np.nan
        test_dict[battery_id][f'mean_grad_last_{k}_cycles'] = mean_slope

In [4]:
sample_battery_id = 'b3c45'
print("battery_id:", test_dict.keys())
features = test_dict[sample_battery_id].keys()
print(f"sample_battery_id ({sample_battery_id}) features:")
for feature in features:
    print(f'    -{feature} : ({type(test_dict[sample_battery_id][feature])})')

battery_id: dict_keys(['b3c0', 'b3c1', 'b3c3', 'b3c4', 'b3c5', 'b3c6', 'b3c7', 'b3c8', 'b3c9', 'b3c10', 'b3c11', 'b3c12', 'b3c13', 'b3c14', 'b3c15', 'b3c16', 'b3c17', 'b3c18', 'b3c19', 'b3c20', 'b3c21', 'b3c22', 'b3c24', 'b3c25', 'b3c26', 'b3c27', 'b3c28', 'b3c29', 'b3c30', 'b3c31', 'b3c33', 'b3c34', 'b3c35', 'b3c36', 'b3c40', 'b3c41', 'b3c42', 'b3c43', 'b3c44', 'b3c45'])
sample_battery_id (b3c45) features:
    -q_d_n : (<class 'list'>)
    -cycle : (<class 'int'>)
    -trimmed_q_d_n : (<class 'list'>)
    -slope_all_cycles : (<class 'float'>)
    -slope_last_10_cycles : (<class 'float'>)
    -slope_last_50_cycles : (<class 'float'>)
    -slope_last_100_cycles : (<class 'float'>)
    -slope_last_200_cycles : (<class 'float'>)
    -slope_last_300_cycles : (<class 'float'>)
    -slope_last_400_cycles : (<class 'float'>)
    -slope_last_500_cycles : (<class 'float'>)
    -slope_last_600_cycles : (<class 'float'>)
    -slope_last_700_cycles : (<class 'float'>)
    -slope_last_800_cycles : 

In [5]:
os.environ["NEO4J_URI"] = "neo4j+s://3b31837b.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "D4W3Zfi44nAJfStBuxSE2DpKhlk_nMP6ybEjvOX5qxw"

from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(refresh_schema=False)

graph.refresh_schema()
print(graph.schema)

/tmp/ipykernel_4310/636819369.py:6: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(refresh_schema=False)


Node properties:
ChargingPolicy {charging_policy: STRING}
Battery {battery_id: STRING, total_cycles: INTEGER, slope_all_cycles: FLOAT, trimmed_q_d_n_avg: FLOAT, name: STRING, slope_last_10_cycles: FLOAT, slope_last_50_cycles: FLOAT, slope_last_100_cycles: FLOAT, slope_last_200_cycles: FLOAT, slope_last_300_cycles: FLOAT, slope_last_400_cycles: FLOAT, slope_last_500_cycles: FLOAT, slope_last_600_cycles: FLOAT, slope_last_700_cycles: FLOAT, slope_last_800_cycles: FLOAT, slope_last_900_cycles: FLOAT, slope_last_1000_cycles: FLOAT, mean_grad_last_10_cycles: FLOAT, mean_grad_last_50_cycles: FLOAT, mean_grad_last_100_cycles: FLOAT, mean_grad_last_200_cycles: FLOAT, mean_grad_last_300_cycles: FLOAT, mean_grad_last_400_cycles: FLOAT, mean_grad_last_500_cycles: FLOAT, mean_grad_last_600_cycles: FLOAT, mean_grad_last_700_cycles: FLOAT, mean_grad_last_800_cycles: FLOAT, mean_grad_last_900_cycles: FLOAT, mean_grad_last_1000_cycles: FLOAT}
Relationship properties:

The relationships:
(:ChargingPoli

## Example Query

In [19]:
from pprint import pprint
pprint(test_dict['b3c3'])

{'cycle': 1116,
 'mean_grad_last_1000_cycles': -0.00018278807401657106,
 'mean_grad_last_100_cycles': -0.0007513692975044251,
 'mean_grad_last_10_cycles': -0.0005299955606460572,
 'mean_grad_last_200_cycles': -0.0006169727444648742,
 'mean_grad_last_300_cycles': -0.0004754801591237386,
 'mean_grad_last_400_cycles': -0.0003807681798934936,
 'mean_grad_last_500_cycles': -0.000319430947303772,
 'mean_grad_last_50_cycles': -0.0007365322113037109,
 'mean_grad_last_600_cycles': -0.0002763838569323222,
 'mean_grad_last_700_cycles': -0.00024350694247654507,
 'mean_grad_last_800_cycles': -0.00021823398768901824,
 'mean_grad_last_900_cycles': -0.00019931296507517496,
 'q_d_n': [1.062273621559143,
           1.0628892183303833,
           1.0632294416427612,
           1.0635850429534912,
           1.064354658126831,
           1.0645473003387451,
           1.0647578239440918,
           1.0649096965789795,
           1.0648976564407349,
           1.0651054382324219,
           1.0649564266204

In [16]:
# ['b3c0', 'b3c1', 'b3c3', 'b3c4', 'b3c5', 'b3c6', 'b3c7', 'b3c8', 'b3c9', 'b3c10', 'b3c11', 'b3c12', 'b3c13', 'b3c14', 'b3c15', 'b3c16', 'b3c17', 'b3c18', 'b3c19', 'b3c20', 'b3c21', 'b3c22', 'b3c24', 'b3c25', 'b3c26', 'b3c27', 'b3c28', 'b3c29', 'b3c30', 'b3c31', 'b3c33', 'b3c34', 'b3c35', 'b3c36', 'b3c40', 'b3c41', 'b3c42', 'b3c43', 'b3c44', 'b3c45']
test_key = 'b3c3'
test_battery = test_dict[test_key]


# Define feature for similarity comparison
feature = "slope_last_500_cycles"
test_slope = test_battery[f"{feature}"]  # Change this to compare different features
print(test_slope)

# Define similarity threshold and number of results
THRESHOLD = 10  # Adjust based on desired similarity
TOP_K = 3  # Number of closest matches to return

# -------- SINGLE QUERY: Find closest batteries & their charging policies -------- #
query = f"""
MATCH (cp:ChargingPolicy)-[:USED_BY]->(b:Battery)
WHERE abs(b.{feature} - $test_slope) < $threshold
RETURN b.battery_id AS battery_id, 
       b.{feature} AS {feature}, 
       abs(b.{feature} - $test_slope) AS similarity,
       cp.charging_policy AS charging_policy
ORDER BY similarity ASC
LIMIT $top_k
"""

# Execute query with parameters
results = graph.query(query, params={"test_slope": test_slope, "threshold": THRESHOLD, "top_k": TOP_K})

# -------- Display Results -------- #
print(f"Test Battery: {test_key} ({feature} = {test_slope})")
print(f"Closest {TOP_K} Matches in Neo4j (within threshold {THRESHOLD}):")

for idx, battery in enumerate(results):
    battery_id = battery["battery_id"]
    battery_feature_value = battery[f"{feature}"]
    similarity = battery["similarity"]
    charging_policy = battery["charging_policy"] if battery["charging_policy"] else "Unknown"

    print(f"{idx+1}. Battery: {battery_id} - {feature}: {battery_feature_value} (Diff: {similarity})")
    print(f"   Charging Policy: {charging_policy}\n")


-0.00031944239139556886
Test Battery: b3c3 (slope_last_500_cycles = -0.00031944239139556886)
Closest 3 Matches in Neo4j (within threshold 10):
1. Battery: b1c3 - slope_last_500_cycles: -0.0003280808925628662 (Diff: 8.638501167297326e-06)
   Charging Policy: 4C(80%)-4C

2. Battery: b1c11 - slope_last_500_cycles: -0.00034225058555603026 (Diff: 2.2808194160461393e-05)
   Charging Policy: 5.4C(50%)-3C

3. Battery: b1c18 - slope_last_500_cycles: -0.00034250414371490476 (Diff: 2.30617523193359e-05)
   Charging Policy: 5.4C(70%)-3C



#TODO LLM pipeline and eval

In [12]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
if os.environ.get("NVIDIA_API_KEY") is None:
    os.environ["NVIDIA_API_KEY"] = 'ZmZpMzhuYTduaG9lMjZjdHB2aTNodmJ2Mzk6ZmQxMTRhODAtMDA2YS00N2IwLWFjODktMmUxMmE2MGMwMDcz'

# Set up NVIDIA ChatNVIDIA LLM
llm = ChatNVIDIA(
    model="mistralai/mixtral-8x22b-instruct-v0.1",
    api_key=os.environ["NVIDIA_API_KEY"],
    temperature=0.5,
    top_p=1,
    max_tokens=1024,
)


In [23]:
import os
from neo4j import GraphDatabase
from langchain.tools import tool
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# Neo4j Database Connection
NEO4J_URI = "neo4j+s://3b31837b.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "D4W3Zfi44nAJfStBuxSE2DpKhlk_nMP6ybEjvOX5qxw"

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Define LangChain Tool
@tool
def query_similar_batteries(test_data ):
    
    """
    Query Neo4j graph DB to find batteries similar to a given test battery based on a feature.
    
    Args:
        test_data (list) : The test data gotten from user

    Returns:
        List[Dict]: A list of the closest matching batteries and their charging policies.
    """
    feature: str = "slope_last_500_cycles"; threshold: int = 10; top_k: int = 3
    with driver.session() as session:
        # # Retrieve the feature value for the test battery
        # test_query = f"MATCH (b:Battery) WHERE b.battery_id = $test_key RETURN b.{feature} AS feature_value"
        # test_result = session.run(test_query, test_key=test_key).single()
        
        # if test_result is None:
        #     return {"error": f"Battery ID '{test_key}' not found in database."}
        
        # test_value = test_result["feature_value"]

        # Similarity Query
        query = f"""
        MATCH (cp:ChargingPolicy)-[:USED_BY]->(b:Battery)
        WHERE abs(b.{feature} - $test_value) < $threshold
        RETURN b.battery_id AS battery_id, 
               b.{feature} AS feature_value, 
               abs(b.{feature} - $test_value) AS similarity,
               cp.charging_policy AS charging_policy
        ORDER BY similarity ASC
        LIMIT $top_k
        """

        results = session.run(query, test_value=test_value, threshold=threshold, top_k=top_k)
        
        output = []
        for record in results:
            output.append({
                "battery_id": record["battery_id"],
                "feature_value": record["feature_value"],
                "similarity": record["similarity"],
                "charging_policy": record["charging_policy"] or "Unknown"
            })
        
        return output



In [25]:
os.environ["NVIDIA_API_KEY"] = "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"

llm = ChatNVIDIA(
    model="mistralai/mixtral-8x22b-instruct-v0.1",
    api_key=os.environ["NVIDIA_API_KEY"],
    temperature=0.5,
    top_p=1,
    max_tokens=2048,
)

# Define tools list for LangChain agent
tools = [query_similar_batteries]

# Create LangChain agent with tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Enables reasoning over the tool
    verbose=True,
)

# Test the agent
response = agent.run('''Find the most similar batteries to [1.062273621559143,
1.0628892183303833,
1.0632294416427612,
1.0635850429534912,
1.064354658126831,
1.0645473003387451,
1.0647578239440918,
1.0649096965789795,
1.0648976564407349,
1.0651054382324219,
1.0649564266204834,
1.06504225730896,
1.0651119947433472,
1.0651532411575317,
1.065083622932434,
1.0650676488876343,
1.0652220249176025,
1.065100073814392,
1.0650522708892822,
1.0653680562973022,
1.0653455257415771,
1.0652103424072266,
1.0651191473007202,
1.0652167797088623,
1.0650957822799683,
1.065032720565796,
1.064998745918274,
1.0652134418487549,
1.0655317306518555,
1.0654642581939697,
1.065534234046936,
1.0654488801956177,
1.0654385089874268,
1.0655080080032349,
1.0654411315917969,
1.0654168128967285,
1.0653618574142456,
1.0652990341186523,
1.0652203559875488,
1.0653626918792725,
1.0651566982269287,
1.065282940864563,
1.0651912689208984,
1.0651483535766602,
1.0651661157608032,
1.0651146173477173,
1.0649980306625366,
1.0650767087936401,
1.0649843215942383,
1.0650570392608643,
1.0649139881134033,
1.0650827884674072,
1.065107822418213,
1.0648400783538818,
1.064950704574585,
1.0648733377456665,
1.0646967887878418,
1.0647985935211182,
1.064778208732605,
1.0647236108779907,
1.064716100692749,
1.0646660327911377,
1.0645791292190552,
1.0646371841430664,
1.0646153688430786,
1.0646387338638306,
1.0646162033081055,
1.0645502805709839,
1.0645064115524292,
1.064363718032837,
1.0644257068634033,
1.0641474723815918,
1.0641118288040161,
1.0640957355499268,
1.0640342235565186,
1.0639188289642334,
1.0639616250991821,
1.0640562772750854,
1.0641353130340576,
1.0640227794647217,
1.0640366077423096,
1.064078450202942,
1.0641419887542725,
1.0640790462493896,
1.0640040636062622,
1.0640980005264282,
1.0639415979385376,
1.0639828443527222,
1.0638678073883057,
1.0640287399291992,
1.064226508140564,
1.0641297101974487,
1.0639591217041016,
1.0638160705566406,
1.0637890100479126,
1.0637491941452026,
1.0636584758758545,
1.0635555982589722,
1.0634772777557373,
1.0635945796966553,
1.0636000633239746,
1.0633846521377563,
1.0635617971420288,
1.0634026527404785,
1.0633569955825806,
1.0636451244354248,
1.0634143352508545,
1.0633677244186401,
1.0634486675262451,
1.0632474422454834,
1.063346028327942,
1.0632696151733398,
1.0632603168487549,
1.0633196830749512,
1.0633500814437866,
1.0633864402770996,
1.0633623600006104,
1.063342809677124,
1.0635364055633545,
1.0633281469345093,
1.063503384590149,
1.0633162260055542,
1.0634745359420776,
1.0633625984191895,
1.0631896257400513,
1.0631864070892334,
1.0630978345870972,
1.0630860328674316,
1.0630956888198853,
1.062948226928711,
1.0630391836166382,
1.0628714561462402,
1.0628944635391235,
1.0627570152282715,
1.0627158880233765,
1.0626405477523804,
1.0626107454299927,
1.0625473260879517,
1.0624624490737915,
1.0624635219573975,
1.0623726844787598,
1.0622832775115967,
1.0623804330825806,
1.0622895956039429,
1.0622154474258423,
1.0622676610946655,
1.0621042251586914,
1.0621674060821533,
1.062416434288025,
1.0624182224273682,
1.0624092817306519,
1.062561273574829,
1.0624380111694336,
1.0625156164169312,
1.0623723268508911,
1.0624722242355347,
1.0623623132705688,
1.0622979402542114,
1.0623260736465454,
1.0623716115951538,
1.0621659755706787,
1.0621401071548462,
1.0621681213378906,
1.0620464086532593,
1.0619442462921143,
1.0616832971572876,
1.0617032051086426,
1.0617008209228516,
1.061718225479126,
1.0614999532699585,
1.0616084337234497,
1.0615086555480957,
1.0616148710250854,
1.0614417791366577,
1.0612794160842896,
1.0613391399383545,
1.0612945556640625,
1.061395287513733,
1.0614631175994873,
1.061232089996338,
1.0612276792526245,
1.061234474182129,
1.0612196922302246,
1.0611668825149536,
1.0609945058822632,
1.0610575675964355,
1.061003565788269,
1.0610605478286743,
1.0608776807785034,
1.060804009437561,
1.0608259439468384,
1.0608209371566772,
1.0607768297195435,
1.0608211755752563,
1.060605525970459,
1.0603759288787842,
1.060418725013733,
1.0604422092437744,
1.0604541301727295,
1.0604467391967773,
1.0603575706481934,
1.0602736473083496,
1.060203194618225,
1.0601608753204346,
1.0602165460586548,
1.0599377155303955,
1.05988347530365,
1.0598288774490356,
1.059796690940857,
1.059897780418396,
1.0599188804626465,
1.0598076581954956,
1.0598783493041992,
1.0599571466445923,
1.0599485635757446,
1.0598273277282715,
1.0599052906036377,
1.0597844123840332,
1.0597200393676758,
1.0598098039627075,
1.0597107410430908,
1.059674620628357,
1.0597447156906128,
1.059546947479248,
1.0595320463180542,
1.0595769882202148,
1.0594489574432373,
1.0593069791793823,
1.059293270111084,
1.0592787265777588,
1.0592595338821411,
1.0591593980789185,
1.0592329502105713,
1.0591201782226562,
1.0591520071029663,
1.059159755706787,
1.059186577796936,
1.0589386224746704,
1.0588587522506714,
1.0587884187698364,
1.058893084526062,
1.0588375329971313,
1.0588159561157227,
1.0587248802185059,
1.0585559606552124,
1.0584404468536377,
1.058329701423645,
1.0583292245864868,
1.0583767890930176,
1.0583467483520508,
1.0583653450012207,
1.0583829879760742,
1.0583586692810059,
1.0583926439285278,
1.058068871498108,
1.0580605268478394,
1.0579960346221924,
1.0579227209091187,
1.057973027229309,
1.0579113960266113,
1.0579345226287842,
1.0578440427780151,
1.0577213764190674,
1.0575281381607056,
1.0576486587524414,
1.057680606842041,
1.057813048362732,
1.0575300455093384,
1.057429552078247,
1.0574668645858765,
1.0574090480804443,
1.0573136806488037,
1.057275652885437,
1.0572923421859741,
1.0572333335876465,
1.0571504831314087,
1.0571576356887817,
1.0569946765899658,
1.0569813251495361,
1.057124137878418,
1.057032585144043,
1.0570268630981445,
1.0567625761032104,
1.0566692352294922,
1.0565686225891113,
1.0565370321273804,
1.0566707849502563,
1.0566056966781616,
1.0564886331558228,
1.0564854145050049,
1.0565413236618042,
1.0565085411071777,
1.0563310384750366,
1.0562695264816284,
1.0562994480133057,
1.0561057329177856,
1.0560252666473389,
1.05592679977417,
1.05586576461792,
1.055930256843567,
1.0559329986572266,
1.0558137893676758,
1.0557188987731934,
1.0557650327682495,
1.0555917024612427,
1.0555849075317383,
1.0555733442306519,
1.0555139780044556,
1.0554970502853394,
1.055521845817566,
1.0554981231689453,
1.0554834604263306,
1.0553513765335083,
1.055319905281067,
1.055405855178833,
1.0551964044570923,
1.0551444292068481,
1.0550907850265503,
1.0551128387451172,
1.0548980236053467,
1.0549492835998535,
1.054875373840332,
1.0549601316452026,
1.0550882816314697,
1.0547840595245361,
1.0546979904174805,
1.0544934272766113,
1.0544682741165161,
1.054361343383789,
1.0544356107711792,
1.054478645324707,
1.0543208122253418,
1.054452657699585,
1.0544016361236572,
1.0543344020843506,
1.0542926788330078,
1.0542914867401123,
1.0542298555374146,
1.0540815591812134,
1.0539088249206543,
1.0539923906326294,
1.053823709487915,
1.0538378953933716,
1.053855299949646,
1.0536555051803589,
1.0537484884262085,
1.0536303520202637,
1.053666114807129,
1.0537413358688354,
1.053515911102295,
1.0534887313842773,
1.0535824298858643,
1.0535763502120972,
1.0533416271209717,
1.0533782243728638,
1.0533114671707153,
1.0534322261810303,
1.053377628326416,
1.0535143613815308,
1.0533665418624878,
1.0532877445220947,
1.053033709526062,
1.053084373474121,
1.0529966354370117,
1.0528795719146729,
1.0529167652130127,
1.0529398918151855,
1.0528035163879395,
1.0527958869934082,
1.0527440309524536,
1.0527645349502563,
1.0526386499404907,
1.0525312423706055,
1.0526697635650635,
1.0525846481323242,
1.0524733066558838,
1.0523768663406372,
1.0523927211761475,
1.0523430109024048,
1.052290439605713,
1.0522184371948242,
1.0523394346237183,
1.0521278381347656,
1.0520217418670654,
1.0521376132965088,
1.0521206855773926,
1.0520381927490234,
1.052134394645691,
1.0520793199539185,
1.0520477294921875,
1.0518853664398193,
1.0517950057983398,
1.0518360137939453,
1.0516775846481323,
1.0517451763153076,
1.0517182350158691,
1.0517438650131226,
1.051708698272705,
1.051706075668335,
1.0516806840896606,
1.0516357421875,
1.0515433549880981,
1.0515481233596802,
1.0515034198760986,
1.0515152215957642,
1.051531195640564,
1.0515516996383667,
1.0516144037246704,
1.0514241456985474,
1.0514343976974487,
1.051316499710083,
1.0511966943740845,
1.0511131286621094,
1.0512902736663818,
1.0512810945510864,
1.05121648311615,
1.0510847568511963,
1.0511764287948608,
1.0509775876998901,
1.0511219501495361,
1.051085114479065,
1.0510082244873047,
1.0509309768676758,
1.0509570837020874,
1.050951361656189,
1.0508966445922852,
1.0509554147720337,
1.05068838596344,
1.0507529973983765,
1.0505567789077759,
1.0505366325378418,
1.0505291223526,
1.0505459308624268,
1.0504237413406372,
1.0505092144012451,
1.050451636314392,
1.050421953201294,
1.0503113269805908,
1.0503239631652832,
1.0503069162368774,
1.0503273010253906,
1.050229549407959,
1.0501141548156738,
1.0502955913543701,
1.0500599145889282,
1.0499812364578247,
1.0499404668807983,
1.0499844551086426,
1.0499297380447388,
1.049870252609253,
1.0497812032699585,
1.049729585647583,
1.0496175289154053,
1.049581527709961,
1.049405813217163,
1.0494248867034912,
1.049405813217163,
1.0494232177734375,
1.0494180917739868,
1.0491920709609985,
1.0493359565734863,
1.049210786819458,
1.0490362644195557,
1.048943281173706,
1.0489827394485474,
1.0488096475601196,
1.048683524131775,
1.0486551523208618,
1.048567295074463,
1.0485068559646606,
1.0484164953231812,
1.0484163761138916,
1.0482406616210938,
1.0482174158096313,
1.0481235980987549,
1.0480397939682007,
1.0480270385742188,
1.0477018356323242,
1.0477811098098755,
1.047771692276001,
1.0476628541946411,
1.047670841217041,
1.047745943069458,
1.0477290153503418,
1.047790765762329,
1.047705054283142,
1.0477207899093628,
1.0478206872940063,
1.0478005409240723,
1.0478698015213013,
1.0477817058563232,
1.0477495193481445,
1.0476562976837158,
1.0474661588668823,
1.0474869012832642,
1.0473791360855103,
1.0472725629806519,
1.0472654104232788,
1.0472136735916138,
1.0470901727676392,
1.046926498413086,
1.0468803644180298,
1.046778917312622,
1.0467965602874756,
1.046773910522461,
1.0466320514678955,
1.0466101169586182,
1.0464470386505127,
1.046513319015503,
1.0463227033615112,
1.0463337898254395,
1.0461127758026123,
1.0461279153823853,
1.0459814071655273,
1.0459662675857544,
1.0460586547851562,
1.0462546348571777,
1.0459351539611816,
1.0459167957305908,
1.0457310676574707,
1.045693039894104,
1.0456979274749756,
1.0456622838974,
1.0455130338668823,
1.045602560043335,
1.0454516410827637,
1.0453680753707886,
1.0454533100128174,
1.045250415802002,
1.0452322959899902,
1.0452563762664795,
1.0450820922851562,
1.0450118780136108,
1.044844627380371,
1.0449409484863281,
1.044818639755249,
1.0448777675628662,
1.0446326732635498,
1.0447447299957275,
1.0445479154586792,
1.0444979667663574,
1.0442873239517212,
1.0442789793014526,
1.0442062616348267,
1.0441797971725464,
1.044191598892212,
1.0441999435424805,
1.0439684391021729,
1.0437535047531128,
1.0438374280929565,
1.0437122583389282,
1.0437906980514526,
1.0437361001968384,
1.043927788734436,
1.043981671333313,
1.043855905532837,
1.0440270900726318,
1.0438158512115479,
1.0437180995941162,
1.0435523986816406,
1.0434541702270508,
1.0437318086624146,
1.0433872938156128,
1.0435041189193726,
1.0433779954910278,
1.0430747270584106,
1.0431630611419678,
1.0431840419769287,
1.043150782585144,
1.042965292930603,
1.0428314208984375,
1.0429555177688599,
1.042944312095642,
1.0428637266159058,
1.0426926612854004,
1.0426379442214966,
1.0424998998641968,
1.0423837900161743,
1.0423357486724854,
1.0422414541244507,
1.0422630310058594,
1.0421333312988281,
1.0421305894851685,
1.0420235395431519,
1.042052984237671,
1.0419399738311768,
1.04189932346344,
1.0416488647460938,
1.0416160821914673,
1.041703701019287,
1.041556477546692,
1.041535496711731,
1.0417094230651855,
1.041506290435791,
1.0415087938308716,
1.0415797233581543,
1.0415072441101074,
1.0413379669189453,
1.0414555072784424,
1.0413967370986938,
1.0412172079086304,
1.0410038232803345,
1.0411598682403564,
1.0410887002944946,
1.0410507917404175,
1.041041374206543,
1.0409215688705444,
1.0409417152404785,
1.0407979488372803,
1.0407052040100098,
1.040582537651062,
1.0407108068466187,
1.0406848192214966,
1.0405281782150269,
1.0405316352844238,
1.0402288436889648,
1.040305256843567,
1.0403167009353638,
1.0401360988616943,
1.0399997234344482,
1.040074348449707,
1.0400148630142212,
1.039976954460144,
1.0397908687591553,
1.0398080348968506,
1.0397021770477295,
1.039824366569519,
1.0396517515182495,
1.0395472049713135,
1.0394206047058105,
1.0394847393035889,
1.0392929315567017,
1.0391860008239746,
1.0390530824661255,
1.0390632152557373,
1.0391994714736938,
1.0390405654907227,
1.0387721061706543,
1.0388423204421997,
1.0387420654296875,
1.0386797189712524,
1.0386887788772583,
1.038631796836853,
1.0386756658554077,
1.0386005640029907,
1.0383718013763428,
1.0384736061096191,
1.0383764505386353,
1.0382031202316284,
1.0381748676300049,
1.038092017173767,
1.037914752960205,
1.0378403663635254,
1.037739634513855,
1.0377740859985352,
1.0376012325286865,
1.0377269983291626,
1.0375065803527832,
1.0374510288238525,
1.0374144315719604,
1.0373185873031616,
1.0372449159622192,
1.0371202230453491,
1.0371731519699097,
1.0369793176651,
1.0370475053787231,
1.0367947816848755,
1.0366716384887695,
1.0365817546844482,
1.0366272926330566,
1.0365612506866455,
1.0365338325500488,
1.036346673965454,
1.036402702331543,
1.0362590551376343,
1.036172866821289,
1.0362573862075806,
1.0361071825027466,
1.035927176475525,
1.0359247922897339,
1.0358268022537231,
1.0356427431106567,
1.0355803966522217,
1.0354608297348022,
1.0354756116867065,
1.0353320837020874,
1.0353882312774658,
1.035312294960022,
1.0352753400802612,
1.0352296829223633,
1.035183310508728,
1.0349721908569336,
1.0350011587142944,
1.0347446203231812,
1.0346343517303467,
1.0346401929855347,
1.0345360040664673,
1.034494161605835,
1.0345509052276611,
1.0343786478042603,
1.034319519996643,
1.0344096422195435,
1.0343525409698486,
1.0341655015945435,
1.0340681076049805,
1.0339716672897339,
1.0338423252105713,
1.0335984230041504,
1.0336291790008545,
1.033579707145691,
1.0334945917129517,
1.0332200527191162,
1.0330175161361694,
1.0329692363739014,
1.033046841621399,
1.0330164432525635,
1.0329068899154663,
1.032938003540039,
1.0327399969100952,
1.0327504873275757,
1.0325630903244019,
1.0326592922210693,
1.0324279069900513,
1.032317042350769,
1.0323399305343628,
1.0322080850601196,
1.0320374965667725,
1.0320402383804321,
1.0319401025772095,
1.0317546129226685,
1.0316277742385864,
1.0316338539123535,
1.0313881635665894,
1.0315433740615845,
1.0312901735305786,
1.03119695186615,
1.0311604738235474,
1.0310496091842651,
1.0311899185180664,
1.0311198234558105,
1.030802845954895,
1.0308947563171387,
1.03084397315979,
1.0306622982025146,
1.0306826829910278,
1.0304994583129883,
1.0303834676742554,
1.0304083824157715,
1.0304319858551025,
1.0303058624267578,
1.0301663875579834,
1.0299338102340698,
1.0299499034881592,
1.0297613143920898,
1.029899001121521,
1.0298047065734863,
1.0296326875686646,
1.0294733047485352,
1.0296059846878052,
1.0294264554977417,
1.0294033288955688,
1.0293225049972534,
1.029208779335022,
1.0289963483810425,
1.0288819074630737,
1.0289727449417114,
1.0286654233932495,
1.0285664796829224,
1.0284161567687988,
1.0284817218780518,
1.0282796621322632,
1.0281938314437866,
1.0282273292541504,
1.028106451034546,
1.0278418064117432,
1.0278304815292358,
1.027729868888855,
1.027722716331482,
1.0277303457260132,
1.0274738073349,
1.027397632598877,
1.0273246765136719,
1.0272486209869385,
1.0270543098449707,
1.0269203186035156,
1.026785969734192,
1.0266751050949097,
1.0265088081359863,
1.0264997482299805,
1.0264884233474731,
1.0261523723602295,
1.0261402130126953,
1.0259554386138916,
1.0258991718292236,
1.025885820388794,
1.025673747062683,
1.0256121158599854,
1.0256541967391968,
1.0254402160644531,
1.02538001537323,
1.0252610445022583,
1.0251446962356567,
1.0249282121658325,
1.0249934196472168,
1.0249098539352417,
1.0249078273773193,
1.0245980024337769,
1.0246214866638184,
1.0244135856628418,
1.0243949890136719,
1.024195909500122,
1.0239505767822266,
1.0237576961517334,
1.023567795753479,
1.0233622789382935,
1.0233579874038696,
1.0232090950012207,
1.023171067237854,
1.0229192972183228,
1.0229557752609253,
1.0227044820785522,
1.0225027799606323,
1.0223063230514526,
1.022331714630127,
1.022062063217163,
1.0220099687576294,
1.0218123197555542,
1.0215866565704346,
1.0214478969573975,
1.0211752653121948,
1.0212403535842896,
1.020951747894287,
1.0208775997161865,
1.0206595659255981,
1.0204559564590454,
1.0204100608825684,
1.0202261209487915,
1.0199944972991943,
1.0199456214904785,
1.0198159217834473,
1.0196572542190552,
1.0195382833480835,
1.01949143409729,
1.0193248987197876,
1.0193027257919312,
1.0192162990570068,
1.019069790840149,
1.0188168287277222,
1.018492341041565,
1.0183777809143066,
1.0181975364685059,
1.0179479122161865,
1.0180355310440063,
1.0176080465316772,
1.0174723863601685,
1.0173301696777344,
1.0171456336975098,
1.0169742107391357,
1.0167661905288696,
1.016601800918579,
1.0163787603378296,
1.016329050064087,
1.0161114931106567,
1.015921711921692,
1.0156502723693848,
1.0154350996017456,
1.015232801437378,
1.0152109861373901,
1.0149332284927368,
1.014704942703247,
1.014557123184204,
1.0143582820892334,
1.014331340789795,
1.014075517654419,
1.0139540433883667,
1.0137543678283691,
1.0137169361114502,
1.0134786367416382,
1.0131676197052002,
1.0129362344741821,
1.0128077268600464,
1.0125346183776855,
1.0123704671859741,
1.0122342109680176,
1.012050986289978,
1.0118223428726196,
1.0115478038787842,
1.0112468004226685,
1.0111953020095825,
1.0109219551086426,
1.010571002960205,
1.0102555751800537,
1.0102277994155884,
1.0099785327911377,
1.0096832513809204,
1.009548306465149,
1.0092921257019043,
1.0089545249938965,
1.0087224245071411,
1.0086082220077515,
1.0083585977554321,
1.008135199546814,
1.0079728364944458,
1.0075490474700928,
1.007307767868042,
1.0070593357086182,
1.006907343864441,
1.0065641403198242,
1.0061612129211426,
1.005934476852417,
1.0056545734405518,
1.0053731203079224,
1.0051684379577637,
1.0047940015792847,
1.0046448707580566,
1.0042558908462524,
1.0039668083190918,
1.0039432048797607,
1.0035300254821777,
1.0031684637069702,
1.0028550624847412,
1.0024993419647217,
1.0023081302642822,
1.0020884275436401,
1.0017262697219849,
1.0012632608413696,
1.001178503036499,
1.0007692575454712,
1.0003851652145386,
1.0001121759414673,
0.9997730851173401,
0.9993571043014526,
0.9991478323936462,
0.9987947344779968,
0.9984691143035889,
0.9981126189231873,
0.997881293296814,
0.9975175857543945,
0.9971339106559753,
0.9968004822731018,
0.996492862701416,
0.9960693717002869,
0.9957124590873718,
0.995343029499054,
0.9951151013374329,
0.9946428537368774,
0.9944179654121399,
0.9940158128738403,
0.9936662912368774,
0.9933477640151978,
0.9928984642028809,
0.992658257484436,
0.9923433065414429,
0.9918461441993713,
0.9913632273674011,
0.99097740650177,
0.9905731081962585,
0.9901652932167053,
0.989654541015625,
0.9892853498458862,
0.9887201189994812,
0.9882497787475586,
0.9878756403923035,
0.9873964190483093,
0.9869825839996338,
0.9866486191749573,
0.986077606678009,
0.9856278896331787,
0.9851554036140442,
0.9847661852836609,
0.9842181205749512,
0.9837288856506348,
0.9830800890922546,
0.9826616644859314,
0.9820739030838013,
0.9815914630889893,
0.9808951616287231,
0.9805153012275696,
0.9797985553741455,
0.9792948961257935,
0.9788104891777039,
0.9780702590942383,
0.9775357246398926,
0.9769321084022522,
0.9763514399528503,
0.9758071899414062,
0.9749801158905029,
0.9745562672615051,
0.973811686038971,
0.9731883406639099,
0.9724988341331482,
0.9717762470245361,
0.9713336825370789,
0.9705810546875,
0.9699580073356628,
0.9694122076034546,
0.9686618447303772,
0.968087375164032,
0.9674360156059265,
0.9667508006095886,
0.9659848213195801,
0.9652893543243408,
0.9647707343101501,
0.9640413522720337,
0.9635429382324219,
0.9626790881156921,
0.9619767069816589,
0.9614003896713257,
0.9607375264167786,
0.960105299949646,
0.9593997001647949,
0.95868980884552,
0.9582237005233765,
0.9574398398399353,
0.9567121267318726,
0.9562148451805115,
0.9553981423377991,
0.9547524452209473,
0.9539196491241455,
0.9533392786979675,
0.9526501297950745,
0.9519836902618408,
0.9512143135070801,
0.9503753185272217,
0.9497585296630859,
0.9491064548492432,
0.9482452273368835,
0.9475560188293457,
0.9468288421630859,
0.9460339546203613,
0.9453936815261841,
0.9446344971656799,
0.9439139366149902,
0.9431259632110596,
0.9424450993537903,
0.9417177438735962,
0.9410921931266785,
0.940294086933136,
0.9395966529846191,
0.9387914538383484,
0.9380810856819153,
0.9372740983963013,
0.9364393949508667,
0.9356092810630798,
0.9348737001419067,
0.9339896440505981,
0.9332987070083618,
0.9325118660926819,
0.9317516684532166,
0.9309090971946716,
0.9300998449325562,
0.929418683052063,
0.9285712242126465,
0.9278254508972168,
0.9269121289253235,
0.9261618256568909,
0.9253892302513123,
0.9246194362640381,
0.9238340854644775,
0.9231447577476501,
0.9222524762153625,
0.9213644862174988,
0.9206831455230713,
0.9197306036949158,
0.9189459085464478,
0.9181061387062073,
0.9170612096786499,
0.9163622856140137,
0.9155522584915161,
0.9146732091903687,
0.9138069748878479,
0.913077175617218,
0.9122402667999268,
0.9114055633544922,
0.9105556011199951,
0.9098386764526367,
0.9090510606765747,
0.9081748723983765,
0.9074442982673645,
0.9067630171775818,
0.9059622287750244,
0.9051644206047058,
0.9043552875518799,
0.9036166667938232,
0.9027262926101685,
0.9019569158554077,
0.9012070894241333,
0.9003502130508423,
0.899665117263794,
0.8988887071609497,
0.8980581164360046,
0.8973342776298523,
0.8964959383010864,
0.8957325220108032,
0.8948907256126404,
0.8942241668701172,
0.8934617638587952,
0.8925890922546387,
0.8917596340179443,
0.8910397291183472,
0.8902354836463928,
0.8895269632339478,
0.8886883854866028,
0.8878942728042603,
0.887153685092926,
0.8864036202430725,
0.8855053186416626,
0.8847479224205017,
0.8839718103408813,
0.8830833435058594,
0.8824843764305115,
0.8817633390426636,
0.8811092972755432,
0.8805840611457825,
0.8805840611457825,
0.8805840611457825] using slope_last_500_cycles, with a threshold of 10 and top 3 matches.''')
print(response)




> Entering new AgentExecutor chain...


Exception: [402] Payment Required
Account 'CCHnburSxTVrryu3zh_QWmQENl2cj0z3aoENMvzncs8': Cloud credits expired - Please contact NVIDIA representatives